In [2]:
!pip install tensorflow-gpu==2.0

     |████████████████████████████████| 380.8MB 48kB/s 
     |████████████████████████████████| 3.8MB 31.7MB/s 
     |████████████████████████████████| 450kB 35.6MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=c670d4f13602cec989383c8eca544a130c6072065455621974822f9ee47b541b
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.3.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorflow-estimator<2.4.0,>=2.3.0, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing ins

Here, I have taken a dataset of cats and dogs as a zip file and unzipped it

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2020-08-10 19:15:37--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 66.102.1.128, 64.233.166.128, 74.125.133.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|66.102.1.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M  54.2MB/s    in 1.2s    

2020-08-10 19:15:38 (54.2 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



Importing all necessary packages

In [4]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.0.0'

In [5]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [6]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [7]:
zip_object.extractall("./")

In [8]:
zip_object.close()

In [9]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [10]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

We set the image shape to be 128x128

In [11]:
IMG_SHAPE = (128, 128, 3)

We set up a base model for our transfer learning and fine tuning. Here the bast model is chosen to be MobileNetV2 with weights as imagenet

In [12]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

9412608/9406464 [==============================] - 1s 0us/step


In [13]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

For transfer learning, we need to freeze the entire base model and we need to train only our customised model. Hence we are making the base model not trainable

In [14]:
base_model.trainable = False

In [22]:
base_model.input

<tf.Tensor 'input_1:0' shape=(None, 128, 128, 3) dtype=float32>

In [15]:
base_model.output

<tf.Tensor 'out_relu/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

We do GlobalAveragePooling2D in order to get the data in the shape we require it



**One major thing to be noted here is that our model is not declared to be sequential. We are passing our previous models to our next model**

In [16]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [17]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Identity:0' shape=(None, 1280) dtype=float32>

In [ ]:
We move on to declaring our dense hidden layers

In [18]:
dense=tf.keras.layers.Dense(units=1000,activation='relu')(global_average_layer)
dropout=tf.keras.layers.Dropout(0.3)(dense)

In [19]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(dropout)

In [20]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [23]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

Binary cross entropy: because we have 2 classes,i.e, cats/dogs

In [24]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

Batch Normalisation

In [25]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

Image data augmentation

In [26]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=40, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [27]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=40, class_mode="binary")

Found 1000 images belonging to 2 classes.


In [28]:
model.fit_generator(train_generator, epochs=10, validation_data=valid_generator)

Epoch 1/10
50/50 [==============================] - 24s 479ms/step - loss: 0.3059 - accuracy: 0.8725 - val_loss: 0.1701 - val_accuracy: 0.9330
Epoch 2/10
50/50 [==============================] - 19s 372ms/step - loss: 0.1706 - accuracy: 0.9355 - val_loss: 0.1661 - val_accuracy: 0.9370
Epoch 3/10
50/50 [==============================] - 19s 370ms/step - loss: 0.1316 - accuracy: 0.9505 - val_loss: 0.2922 - val_accuracy: 0.8980
Epoch 4/10
50/50 [==============================] - 18s 368ms/step - loss: 0.1298 - accuracy: 0.9440 - val_loss: 0.1937 - val_accuracy: 0.9300
Epoch 5/10
50/50 [==============================] - 19s 374ms/step - loss: 0.1010 - accuracy: 0.9635 - val_loss: 0.1140 - val_accuracy: 0.9570
Epoch 6/10
50/50 [==============================] - 18s 370ms/step - loss: 0.1056 - accuracy: 0.9595 - val_loss: 0.1231 - val_accuracy: 0.9560
Epoch 7/10
50/50 [==============================] - 19s 376ms/step - loss: 0.0695 - accuracy: 0.9775 - val_loss: 0.3014 - val_accuracy: 0.9120

In [29]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [30]:
valid_accuracy

0.959

**transfer learning is done**

Now we move to fine tuning. For that we need to set the latter part of base model to be trainable. Hence we unfreeze it.

In [31]:
base_model.trainable = True

In [32]:
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 155


This is the number up to which we free it. In the total 155 we are going to freeze till 90

In [33]:
fine_tune_at = 90

Looping to freeze

In [34]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

Compile and fit

In [35]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.00015),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [36]:
model.fit_generator(train_generator,  
                    epochs=10, 
                    validation_data=valid_generator)

Epoch 1/10
50/50 [==============================] - 23s 468ms/step - loss: 0.1191 - accuracy: 0.9540 - val_loss: 0.2701 - val_accuracy: 0.9370
Epoch 2/10
50/50 [==============================] - 22s 438ms/step - loss: 0.0372 - accuracy: 0.9900 - val_loss: 0.4666 - val_accuracy: 0.9080
Epoch 3/10
50/50 [==============================] - 22s 436ms/step - loss: 0.0297 - accuracy: 0.9910 - val_loss: 0.3196 - val_accuracy: 0.9450
Epoch 4/10
50/50 [==============================] - 22s 434ms/step - loss: 0.0200 - accuracy: 0.9940 - val_loss: 0.3599 - val_accuracy: 0.9490
Epoch 5/10
50/50 [==============================] - 22s 434ms/step - loss: 0.0137 - accuracy: 0.9960 - val_loss: 0.4577 - val_accuracy: 0.9340
Epoch 6/10
50/50 [==============================] - 22s 437ms/step - loss: 0.0152 - accuracy: 0.9950 - val_loss: 0.2421 - val_accuracy: 0.9600
Epoch 7/10
50/50 [==============================] - 22s 437ms/step - loss: 0.0082 - accuracy: 0.9975 - val_loss: 0.5173 - val_accuracy: 0.9420

In [37]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [38]:
valid_accuracy

0.943